# Using a Jupyter Notebook for data analysis

Why do this?  
Can run code, keep notes and generate graphical and statistical analyses all in the one spot.    
Backs up nicely on Github and is viewable there.   
Can duplicate the notebook to run the same exact analysis on new data.  

Can use all the resources of the python/R community.  
For example, here we're importing a set of data analysis modules:  

In [ ]:
# Analysis modules
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm
np.set_printoptions(precision=5, suppress=True)  # suppress scientific floatation 
sns.set(color_codes=True)
%matplotlib inline

In [ ]:
from scipy import stats
from scipy.stats import ttest_ind
from statsmodels.formula.api import ols
from sklearn.preprocessing import scale
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

In [ ]:
#to enable looking at files
from IPython.display import FileLinks, FileLink

# to allow changing directories
import os
import functools
from functools import partial
from os import chdir

## Colour the cells!

Yellow
<div class="alert alert-block alert-warning">

Green
<div class = "alert alert-success">

Blue
<div class = "alert alert-info">

Red 
<div class = "alert alert-danger">
Example coloured cell - useful to note cells you need to go back to

Insert image using markdown

![Taipei_bioinformatics](pic.JPG)

## Using the command line to manage data files

What's in the folder?'

In [ ]:
! ls

a bit more detail

In [ ]:
! ls -ltr

### Checking the file  
Checking length  

In [ ]:
! wc -l Iris.txt

Checking content

In [ ]:
! head -3 Iris.txt

In [ ]:
! tail -10 Iris.txt

Does it contain data from species I. setosa?  
"grep" - **g**lobally search a **r**egular **e**xpression and **p**rint

In [ ]:
! grep -c "setosa" Iris.txt

Does it contain data from species I. setosa from Roadside sites?  
"|" is a pipe - it shuttles the data stream from the expression on the left directly to that on the right.  

Helps prevent a bilion tmp files clttering your folder

In [ ]:
! grep "setosa" Iris.txt | grep -c "Roadside"

Get a list of the  species in the file and the combinations of species and sites.  
"cat" concatenates files.  Here it's concatenting it to the command line.  
Use "less" to scroll through a file (doesn't work well in jupyter cells).  

"sort" can sort in a number of different ways (-r reverse order, -n numerically)  
"uniq" removes identical ADJACENT lines (have to sort first)

In [ ]:
! cut -f 5 Iris.txt | sort | uniq > Species
! cat Species

In [ ]:
! cut -f 5,6 Iris.txt | sort | uniq > Species_sites
! cat Species_sites

Count the occurances of each species in the file and summarise.
The while loop takes input from a list (in this case "Species") and does the same operation (grep counting it's occurance in the Iris datasheet) on each one.  The results are written to "Counts".  
">" would write over "Counts" with each opperation.  
">>" APPENDS the output to the file.  

In [ ]:
! while read f ; do grep -c "$f" Iris.txt >> counts ; done < Species

In [ ]:
! cat counts

"Paste" sticks files together side-by-side.  
Useful for adding columns to text files

In [ ]:
! paste Species counts > summary
! cat summary

Infomation on command line tools can be got from the man pages.  
BUT they are not available on the noteable server!  

In [138]:
! man join


JOIN(1)                   BSD General Commands Manual                  JOIN(1)

NNAAMMEE
     jjooiinn -- relational database operator

SSYYNNOOPPSSIISS
     jjooiinn [--aa _f_i_l_e___n_u_m_b_e_r | --vv _f_i_l_e___n_u_m_b_e_r] [--ee _s_t_r_i_n_g] [--oo _l_i_s_t] [--tt _c_h_a_r]
          [--11 _f_i_e_l_d] [--22 _f_i_e_l_d] _f_i_l_e_1 _f_i_l_e_2

DDEESSCCRRIIPPTTIIOONN
     The jjooiinn utility performs an ``equality join'' on the specified files and
     writes the result to the standard output.  The ``join field'' is the
     field in each file by which the files are compared.  The first field in
     each line is used by default.  There is one line in the output for each
     pair of lines in _f_i_l_e_1 and _f_i_l_e_2 which have identical join fields.  Each
     output line consists of the join field, the remaining fields from _f_i_l_e_1
     and then the remaining

Change the spelling of a species name.  
"sed" is a **s**tream text **ed**itor.  
It's a mini programming language.  I use it only for this!  

In [ ]:
! sed 's/versicolor/versacolor/g' Iris.txt > tmp.txt

In [ ]:
! head -10 tmp.txt

Change the species name only if it is a marsh location.  

In [ ]:
 # substitute "setosa" with "palustra" ONLY for lines which contain "Marsh"
! awk '/Marsh/{gsub(/setosa/, "palustra")}; 1' Iris.txt > marsh.txt
! grep "Marsh" marsh.txt | tail

"awk", like sed is an early programming language.  
It's very powerful and there is a great stash of oneliners for file management.  
https://www.pement.org/awk/awk1line.txt

In [ ]:
# delete trailing whitespace (spaces, tabs) from end of each line
 awk '{sub(/[ \t]+$/, "")};1'
    
# change "scarlet" or "ruby" or "puce" to "red"
 awk '{gsub(/scarlet|ruby|puce/, "red")}; 1'
    
 # delete ALL blank lines from a file
 awk NF
 awk '/./'

 # print only lines of 65 characters or longer
 awk 'length > 64'

## Using Python for data anlysis

Read in from datasheet:

In [ ]:
df = pd.read_table("Iris.txt")

Check size of dataframe, gives list with number of rows and number of columns

In [ ]:
df.shape

In [ ]:
df.head(3)

Check the metrics

In [ ]:
df.describe()

Look at the relationships between variables

In [ ]:
sns.pairplot(df);

Fancier mapping.
Look at 
https://seaborn.pydata.org/tutorial/distributions.html#visualizing-pairwise-relationships-in-a-dataset
for some great examples

In [ ]:
sns.set(style="darkgrid")

# Subset the iris dataset by species
setosa = df.query("Species == 'setosa'")
virginica = df.query("Species == 'virginica'")

# Set up the figure
f, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect("equal")

# Draw the two density plots
ax = sns.kdeplot(setosa.sepal_width, setosa.sepal_length,
                 cmap="Reds", shade=True, shade_lowest=False)
ax = sns.kdeplot(virginica.sepal_width, virginica.sepal_length,
                 cmap="Blues", shade=True, shade_lowest=False)

# Add labels to the plot
red = sns.color_palette("Reds")[-2]
blue = sns.color_palette("Blues")[-2]
ax.text(2.5, 8.2, "virginica", size=16, color=blue)
ax.text(3.8, 4.5, "setosa", size=16, color=red)


Does Site seem to affect any of the metrics??

In [ ]:
site_colors={'Field': 'green', 'Marsh': 'blue', 'Roadside': 'black'}

In [ ]:
sns.pairplot(df, vars=['petal_width', 'sepal_width', 'petal_length', 'sepal_length'], palette = site_colors, kind='reg', hue='Site')

No obvious effect of Site, but there are clearly several different groups.  Plot by Species

In [ ]:
sns.pairplot(df, vars=['petal_width', 'sepal_width', 'petal_length', 'sepal_length'], kind='reg', hue='Species')

Get the means by species

In [ ]:
df.groupby('Species').mean()

Compare distribution of metrics by species

In [ ]:
sns.boxplot(x="Species", y="petal_length", data=df);

Pile up all the numerical values by Species and plot

In [ ]:
df2 = pd.melt(df.loc[:,"sepal_length":"Species"], "Species", var_name="measurement")

In [ ]:
sns.swarmplot(x="measurement", y="value", hue="Species",
              palette=["r", "c", "y"], data=df2)

Compare to sorted by site

In [ ]:
df3 = pd.melt(df.loc[:,["sepal_length", "sepal_width", "petal_length", "petal_width","Site"]], "Site", var_name="measurement")
sns.swarmplot(x="measurement", y="value", hue="Site",
              palette=["r", "c", "y"], data=df3)

Test for significance of the differences between Species in petal_length
Get the sets of data for each species

In [ ]:
versicolor = df[df['Species']=='versicolor']
virginica = df[df['Species']=='virginica']
setosa = df[df['Species']=='setosa']

### T-test

Is sepal length the same for *I. veriscolor* and *I. virginia*?

In [ ]:
scipy.stats.ttest_ind(versicolor['sepal_length'], virginica['sepal_length'], equal_var=False)

Is sepal length the same for *I. setosa* and *I. virginia*?

In [ ]:
scipy.stats.ttest_ind(setosa['sepal_length'], virginica['sepal_length'], equal_var=False)

### Is there a relationship between sepal length and petal length by species  - using ordinary least squares

See more discussion at:
https://www.datarobot.com/blog/ordinary-least-squares-in-python/

In [ ]:
model = ols('sepal_length ~ Species*petal_length', df).fit()
print(model.summary())

### What the metrics are:

The left part of the first table provides basic information about the model fit:  

Dep. Variable	Which variable is the response in the model  
Model	What model you are using in the fit  
Method	How the parameters of the model were calculated  
No. Observations	The number of observations (examples)  
DF Residuals	Degrees of freedom of the residuals. Number of observations - number of parameters  
DF Model	Number of parameters in the model (not including the constant term if present)  

The right part of the first table shows the goodness of fit
Element	Description  
R-squared	The coefficient of determination. A statistical measure of how well the regression line approximates the real data points  
Adj. R-squared	The above value adjusted based on the number of observations and the degrees-of-freedom of the residuals  
F-statistic	A measure how significant the fit is. The mean squared error of the model divided by the mean squared error of the residuals  
Prob (F-statistic)	The probability that you would get the above statistic, given the null hypothesis that they are unrelated  
Log-likelihood	The log of the likelihood function.  
AIC	The Akaike Information Criterion. Adjusts the log-likelihood based on the number of observations and the complexity of the model.  
BIC	The Bayesian Information Criterion. Similar to the AIC, but has a higher penalty for models with more parameters.  

The second table reports for each of the coefficients
Description The name of the term in the model  
coef	The estimated value of the coefficient  
std err	The basic standard error of the estimate of the coefficient. More sophisticated errors are also available.  
t	The t-statistic value. This is a measure of how statistically significant the coefficient is.  
P > |t|	P-value that the null-hypothesis that the coefficient = 0 is true. If it is less than the confidence level, often 0.05, it indicates that there is a statistically significant relationship between the term and the response.  
[95.0% Conf. Interval]	The lower and upper values of the 95% confidence interval  

Finally, there are several statistical tests to assess the distribution of the residuals
Skewness	A measure of the symmetry of the data about the mean. Normally-distributed errors should be symmetrically distributed about the mean (equal amounts above and below the line).  
Kurtosis	A measure of the shape of the distribution. Compares the amount of data close to the mean with those far away from the mean (in the tails).  
Omnibus	D'Angostino's test. It provides a combined statistical test for the presence of skewness and kurtosis.  
Prob(Omnibus)	The above statistic turned into a probability  
Jarque-Bera	A different test of the skewness and kurtosis  
Prob (JB)	The above statistic turned into a probability  
Durbin-Watson	A test for the presence of autocorrelation (that the errors are not independent.) Often important in time-series analysis  
Cond. No	A test for multicollinearity (if in a fit with multiple parameters, the parameters are related with each other).  


### Is there a relationship between Site and Species for floral metrics?

In [ ]:
model = ols('petal_length ~ Site + Species + Site*Species', df).fit()
print(model.summary())

## Correlation heat map
Look at the correalations of the floral metrics.  Are petal width and petal length more closely related than petal length and sepal length?

In [ ]:
# calculate the correlation matrix
corr = df.corr()

# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

Sepal width is quite independent from the other metrics

### Clustering (and some data manipulations) 
Very useful for exploring data  
How do the species and sites cluster?

Expects data in the form  
        var1 var2 var3 var4  
Class1  
Class2  
Class3  

So we need to summarise the dataframe

In [ ]:
df.head(2)

We want to end up with:

| Species_site | av_sepal_length | av_sepal_width | av_petal_length | av_petal_width |
| --- | --- | --- | --- | ---|
|veriscolor_Field | --- | --- | --- | ---| 
|vericolor_Marsh  | --- | --- | --- | ---|
|veriscolor_Road  | --- | --- | --- | ---|
|virginica_Field  | --- | --- | --- | ---|
|virginica_Marsh  | --- | --- | --- | ---|
|virginica_Road  | --- | --- | --- | ---|
|setosa_Field  | --- | --- | --- | ---|
|setosa_Marsh  | --- | --- | --- | ---|
|setosa_Road  | --- | --- | --- | ---|

In [ ]:
bygroup = df.groupby(['Species', 'Site'])

Check that this is what we expect it to be!

In [ ]:
bygroup['petal_length'].describe()

Get the means

In [ ]:
df_av = bygroup['petal_length', 'petal_width', 'sepal_width', 'sepal_width'].mean().reset_index()

In [ ]:
df_av.head(2)

NOTE these are AVERAGES - have to rename the columns, then export the dataframe

In [ ]:
df_av.columns = ['Species', 'Site', 'petal_length_mean', 'petal_width_mean', 'sepal_length_mean', 'sepal_width_mean']
df_av.to_csv('Iris_average.csv', index=False)

Combine the first two columns into one

In [ ]:
df_av["Plant"] = df_av["Species"].map(str) + "_" + df_av["Site"]

Drop the redundant columns,   
Re-order,  
Make the first colum the index

In [ ]:
df_av.drop(["Species","Site"], 1)

In [ ]:
df_av = df_av[['Plant', 'petal_length_mean', 'petal_width_mean', 'sepal_length_mean', 'sepal_width_mean']]

In [ ]:
df_av.head(2)

We can illustrate this data set as a heat map (coloured by absolute value, not correlations as before):

In [ ]:
df_cluster = df_av.set_index('Plant')

In [ ]:
sns.heatmap(df_cluster, annot=True)

In [ ]:
# generate the linkage matrix
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

Z = linkage(df_cluster, 'ward')
c, coph_dists = cophenet(Z, pdist(df_cluster))
c

In [ ]:
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram of accessions by sequence distance')
plt.xlabel('Accession')
plt.ylabel('100*(average(% ID/length of match)/bp in assembly)')
dendrogram(
    Z,
    labels=(df_cluster.index),
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=14.,  # font size for the x axis labels
)
plt.show()

Shows very little varation between sites (short branches),  
no consistent realtionship between sites within a species 
and a closer realtionship between veriscolor and virginica than with setosa

   Read the docs:  
   https://pandas.pydata.org/pandas-docs/stable/10min.html
   
   DataQuest Tutorial on Data Analysis:   
   https://www.dataquest.io/blog/pandas-python-tutorial/
    
   Seaborn:  
   https://seaborn.pydata.org/
    